Topic 5: Hashing & Hash Tables

Task 1: Implementing a Custom Hash Table with Collision Handling

In [1]:
class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.next = None

class HashTableChaining:
    def __init__(self, size=100):
        self.size = size
        self.table = [None] * size

    def _hash(self, key):
        return hash(key) % self.size

    def insert(self, key, value):
        idx = self._hash(key)
        if self.table[idx] is None:
            self.table[idx] = Node(key, value)
        else:
            curr = self.table[idx]
            while True:
                if curr.key == key:
                    curr.value = value  # Update
                    return
                if curr.next is None:
                    break
                curr = curr.next
            curr.next = Node(key, value)

    def get(self, key):
        idx = self._hash(key)
        curr = self.table[idx]
        while curr:
            if curr.key == key:
                return curr.value
            curr = curr.next
        return None

    def delete(self, key):
        idx = self._hash(key)
        curr = self.table[idx]
        prev = None
        while curr:
            if curr.key == key:
                if prev:
                    prev.next = curr.next
                else:
                    self.table[idx] = curr.next
                return True
            prev = curr
            curr = curr.next
        return False

    def display(self):
        for i, node in enumerate(self.table):
            items = []
            while node:
                items.append(f"({node.key}: {node.value})")
                node = node.next
            if items:
                print(f"Index {i}: " + " -> ".join(items))


In [3]:
class HashTableLinearProbing:
    def __init__(self, size=100):
        self.size = size
        self.keys = [None] * size
        self.values = [None] * size

    def _hash(self, key):
        return hash(key) % self.size

    def insert(self, key, value):
        idx = self._hash(key)
        start_idx = idx
        while self.keys[idx] is not None and self.keys[idx] != key:
            idx = (idx + 1) % self.size
            if idx == start_idx:
                raise Exception("Hash table is full")
        self.keys[idx] = key
        self.values[idx] = value

    def get(self, key):
        idx = self._hash(key)
        start_idx = idx
        while self.keys[idx] is not None:
            if self.keys[idx] == key:
                return self.values[idx]
            idx = (idx + 1) % self.size
            if idx == start_idx:
                break
        return None

    def delete(self, key):
        idx = self._hash(key)
        start_idx = idx
        while self.keys[idx] is not None:
            if self.keys[idx] == key:
                self.keys[idx] = None
                self.values[idx] = None
                return True
            idx = (idx + 1) % self.size
            if idx == start_idx:
                break
        return False

    def display(self):
        for i, key in enumerate(self.keys):
            if key is not None:
                print(f"Index {i}: ({key}: {self.values[i]})")


In [4]:
# Test for Chaining
print("Testing Chaining:")
chain_ht = HashTableChaining(10)
chain_ht.insert("apple", 10)
chain_ht.insert("banana", 20)
chain_ht.insert("apple", 15)  # Update
print("Get apple:", chain_ht.get("apple"))  # 15
chain_ht.delete("banana")
chain_ht.display()

# Test for Linear Probing
print("\nTesting Linear Probing:")
linear_ht = HashTableLinearProbing(10)
linear_ht.insert("apple", 10)
linear_ht.insert("banana", 20)
linear_ht.insert("apple", 15)  # Update
print("Get apple:", linear_ht.get("apple"))  # 15
linear_ht.delete("banana")
linear_ht.display()


Testing Chaining:
Get apple: 15
Index 7: (apple: 15)

Testing Linear Probing:
Get apple: 15
Index 7: (apple: 15)


In [5]:
import time
import random
import string

def generate_random_string(length=5):
    return ''.join(random.choices(string.ascii_letters, k=length))

def test_performance():
    keys = [generate_random_string() for _ in range(1000)]
    values = [random.randint(1, 1000) for _ in range(1000)]

    # Chaining
    chaining_ht = HashTableChaining(200)
    start = time.time()
    for k, v in zip(keys, values):
        chaining_ht.insert(k, v)
    for k in keys:
        chaining_ht.get(k)
    chaining_time = time.time() - start

    # Linear Probing
    linear_ht = HashTableLinearProbing(2000)
    start = time.time()
    for k, v in zip(keys, values):
        linear_ht.insert(k, v)
    for k in keys:
        linear_ht.get(k)
    linear_time = time.time() - start

    print(f"\nPerformance (1000 keys):")
    print(f"Chaining Time: {chaining_time:.6f} seconds")
    print(f"Linear Probing Time: {linear_time:.6f} seconds")

test_performance()



Performance (1000 keys):
Chaining Time: 0.004240 seconds
Linear Probing Time: 0.002997 seconds


Task 2: Checking if Two Strings Are Anagrams Using Hashing

In [6]:
def are_anagrams(str1, str2):
    # Step 1: Check if lengths are the same
    if len(str1) != len(str2):
        return False

    # Step 2: Count character frequencies in str1
    count = {}
    for char in str1:
        count[char] = count.get(char, 0) + 1

    # Step 3: Decrease counts based on str2
    for char in str2:
        if char not in count or count[char] == 0:
            return False
        count[char] -= 1

    return True


In [7]:
# Basic test cases
print(are_anagrams("listen", "silent"))  # True
print(are_anagrams("hello", "world"))    # False
print(are_anagrams("triangle", "integral"))  # True
print(are_anagrams("rat", "car"))        # False

# Case sensitivity
print(are_anagrams("Listen", "Silent"))  # False (case-sensitive)

# Large strings
a = "a" * 100000 + "b" * 100000 + "c" * 100000
b = "c" * 100000 + "b" * 100000 + "a" * 100000
print(are_anagrams(a, b))  # True


True
False
True
False
False
True


Task 3: Implementing a Simple Caching Mechanism Using Hash Maps

In [9]:
class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.prev = None
        self.next = None

class LRUCache:
    def __init__(self, capacity):
        self.capacity = capacity
        self.cache = {}  # key: node
        self.head = Node(0, 0)  # Dummy head
        self.tail = Node(0, 0)  # Dummy tail
        self.head.next = self.tail
        self.tail.prev = self.head

    def _remove(self, node):
        """Remove node from the list"""
        prev_node = node.prev
        next_node = node.next
        prev_node.next = next_node
        next_node.prev = prev_node

    def _add_to_front(self, node):
        """Add node right after head"""
        node.next = self.head.next
        node.prev = self.head
        self.head.next.prev = node
        self.head.next = node

    def get(self, key):
        if key in self.cache:
            node = self.cache[key]
            self._remove(node)
            self._add_to_front(node)
            return node.value
        return -1

    def put(self, key, value):
        if key in self.cache:
            self._remove(self.cache[key])
        elif len(self.cache) >= self.capacity:
            # Remove least recently used node
            lru = self.tail.prev
            self._remove(lru)
            del self.cache[lru.key]

        new_node = Node(key, value)
        self._add_to_front(new_node)
        self.cache[key] = new_node

    def display(self):
        node = self.head.next
        result = {}
        while node != self.tail:
            result[node.key] = node.value
            node = node.next
        print("Cache state:", result)


In [10]:
# Test case
cache = LRUCache(5)
cache.put(1, "A")
cache.put(2, "B")
cache.put(3, "C")
cache.put(4, "D")
cache.put(5, "E")
cache.get(2)        # Access 2 -> MRU
cache.put(6, "F")   # Evicts 1
cache.get(4)        # Access 4 -> MRU
cache.put(7, "G")   # Evicts 3
cache.put(8, "H")   # Evicts 5
cache.put(9, "I")   # Evicts 6
cache.get(2)        # Access 2 -> MRU
cache.put(10, "J")  # Evicts 7

cache.display()


Cache state: {10: 'J', 9: 'I', 8: 'H', 7: 'G', 4: 'D'}
